In [1]:
import tensorflow as tf
from tensorflow.python.platform import flags

# from nets.pose_net import pose_exp_net
# from monodepth_model import disp_godard
from nets.pwc_flow import construct_model_pwc_full, feature_pyramid_flow
# from nets.pwc_disp import feature_pyramid_disp
# from optical_flow_warp_fwd import transformerFwd
# from optical_flow_warp_old import transformer_old
from monodepth_dataloader import get_multi_scale_intrinsics
# from utils import inverse_warp, inverse_warp_new
# from loss_utils import SSIM, deprocess_image, preprocess_image,\
#   cal_grad2_error_mask, charbonnier_loss, cal_grad2_error

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
dataloader = MonodepthDataloader()

NameError: name 'MonodepthDataloader' is not defined

In [7]:
opt = flags.FLAGS

In [8]:
class Model_flow(object):
    def __init__(self,
                 image1=None,
                 image2=None,
                 image1r=None,
                 image2r=None,
                 cam2pix=None,
                 pix2cam=None,
                 reuse_scope=False,
                 scope=None):
        summaries = []

        batch_size, H, W, color_channels = map(int, image1.get_shape()[0:4])

        with tf.compat.v1.variable_scope(scope, reuse=reuse_scope):
            feature1 = feature_pyramid_flow(image1, reuse=False)
            feature2 = feature_pyramid_flow(image2, reuse=True)

            optical_flows = construct_model_pwc_full(image1, image2, feature1,
                                                     feature2)

        with tf.compat.v1.variable_scope(scope, reuse=True):
            optical_flows_rev = construct_model_pwc_full(image2, image1,
                                                         feature2, feature1)

        occu_masks = [
            tf.clip_by_value(
                transformerFwd(
                    tf.ones(
                        shape=[batch_size, H / (2**s), W / (2**s), 1],
                        dtype='float32'),
                    flowr, [H / (2**s), W / (2**s)]),
                clip_value_min=0.0,
                clip_value_max=1.0)
            for s, flowr in enumerate(optical_flows_rev)
        ]

        pixel_loss_depth = 0
        pixel_loss_optical = 0
        exp_loss = 0
        flow_smooth_loss = 0
        tgt_image_all = []
        src_image_all = []
        proj_image_depth_all = []
        proj_error_depth_all = []
        exp_mask_stack_all = []
        flyout_map_all = []

        for s in range(opt.num_scales):
            # Scale the source and target images for computing loss at the 
            # according scale.
            curr_tgt_image = tf.image.resize(
                image1,
                [int(opt.img_height / (2**s)), int(opt.img_width / (2**s))], method=tf.image.ResizeMethod.AREA)
            curr_src_image = tf.image.resize(
                image2,
                [int(opt.img_height / (2**s)), int(opt.img_width / (2**s))], method=tf.image.ResizeMethod.AREA)

            occu_mask = occu_masks[s]
            occu_mask_avg = tf.reduce_mean(input_tensor=occu_mask)

            curr_proj_image_optical = transformer_old(
                curr_src_image, optical_flows[s], [H / (2**s), W / (2**s)])
            curr_proj_error_optical = tf.abs(curr_proj_image_optical -
                                             curr_tgt_image)
            pixel_loss_optical += (1.0 - opt.ssim_weight) * tf.reduce_mean(
                input_tensor=curr_proj_error_optical * occu_mask) / occu_mask_avg

            curr_flyout_map = occu_mask

            if opt.ssim_weight > 0:
                pixel_loss_optical += opt.ssim_weight * tf.reduce_mean(
                    input_tensor=SSIM(curr_proj_image_optical * occu_mask, curr_tgt_image *
                         occu_mask)) / occu_mask_avg

            flow_smooth_loss += opt.flow_smooth_weight * cal_grad2_error(
                optical_flows[s] / 20.0, curr_tgt_image, 1.0)

            tgt_image_all.append(curr_tgt_image)
            src_image_all.append(curr_src_image)
            proj_image_depth_all.append(curr_proj_image_optical)
            proj_error_depth_all.append(curr_proj_error_optical)

            flyout_map_all.append(curr_flyout_map)

        self.loss = (pixel_loss_optical + flow_smooth_loss)

        summaries.append(tf.compat.v1.summary.scalar("total_loss", self.loss))
        summaries.append(
            tf.compat.v1.summary.scalar("pixel_loss_depth", pixel_loss_depth))
        summaries.append(
            tf.compat.v1.summary.scalar("pixel_loss_optical", pixel_loss_optical))
        summaries.append(tf.compat.v1.summary.scalar("exp_loss", exp_loss))
        tf.compat.v1.summary.image('scale%d_target_image' % s, \
                         deprocess_image(tgt_image_all[s]))
        tf.compat.v1.summary.image('scale%d_src_image' % s, \
                         deprocess_image(src_image_all[s]))

        tf.compat.v1.summary.image('scale_projected_image',
                         deprocess_image(proj_image_depth_all[s]))
        tf.compat.v1.summary.image('scale_proj_error_error', proj_error_depth_all[s])
        tf.compat.v1.summary.image('scale_flyout_mask', flyout_map_all[s])

        self.summ_op = tf.compat.v1.summary.merge(summaries)



In [9]:
class Model_eval_flow(object):
    def __init__(self, scope=None):
        with tf.compat.v1.variable_scope(scope, reuse=True):
            input_uint8_1 = tf.compat.v1.placeholder(
                tf.uint8, [1, opt.img_height, opt.img_width, 3],
                name='raw_input_1')
            input_uint8_1r = tf.compat.v1.placeholder(
                tf.uint8, [1, opt.img_height, opt.img_width, 3],
                name='raw_input_1r')
            input_uint8_2 = tf.compat.v1.placeholder(
                tf.uint8, [1, opt.img_height, opt.img_width, 3],
                name='raw_input_2')
            input_uint8_2r = tf.compat.v1.placeholder(
                tf.uint8, [1, opt.img_height, opt.img_width, 3],
                name='raw_input_2r')
            input_intrinsic = tf.compat.v1.placeholder(tf.float32, [3, 3])

            cam2pix, pix2cam = get_multi_scale_intrinsics(input_intrinsic,
                                                          opt.num_scales)
            cam2pix = tf.expand_dims(cam2pix, axis=0)
            pix2cam = tf.expand_dims(pix2cam, axis=0)

            input_1 = preprocess_image(input_uint8_1)
            input_2 = preprocess_image(input_uint8_2)
            input_1r = preprocess_image(input_uint8_1r)
            input_2r = preprocess_image(input_uint8_2r)

            feature1 = feature_pyramid_flow(input_1, reuse=True)
            feature2 = feature_pyramid_flow(input_2, reuse=True)

            optical_flows = construct_model_pwc_full(input_1, input_2,
                                                     feature1, feature2)

        self.input_1 = input_uint8_1
        self.input_2 = input_uint8_2
        self.input_r = input_uint8_1r
        self.input_2r = input_uint8_2r
        self.input_intrinsic = input_intrinsic
        self.pred_flow_optical = optical_flows[0]

        # Placeholder created for interface consistency
        self.pred_flow_rigid = tf.constant(0.0)
        self.pred_disp = tf.constant(0.0)
        self.pred_disp2 = tf.constant(0.0)
        self.pred_mask = tf.constant(0.0)

